In [ ]:
# Basic Tutorial on how to use Langchain & PineCone Vector store 

In [1]:
import os 
os.environ["OPENAI_API_KEY"] ="your open ai key" 

In [ ]:
! curl -o data.csv https://healthcaredatasetbucket.s3.amazonaws.com/HealthCare.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30.5M  100 30.5M    0     0  55.7M      0 --:--:-- --:--:-- --:--:-- 55.7M


In [ ]:
import pandas as pd 
df=pd.read_csv('data.csv')[:200]

In [ ]:
output=''.join(df['Answer'].values)

In [ ]:
with open('health.text','w') as f:
    f.write(output)

In [ ]:
! mkdir myfiles 

In [ ]:
! mv /content/health.text /content/myfiles/health.text

In [2]:
!pip install langchain openai -q
!pip install unstructured -q
!pip install pinecone-client -q
!pip install tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.9/709.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.2 MB/s eta 0:00:0

In [3]:
import langchain

print(langchain.__version__)

0.0.154


In [5]:
import os
import openai
import pinecone
from langchain.document_loaders import DirectoryLoader , TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
directory = '/content/myfiles'

loader = DirectoryLoader(directory,loader_cls=TextLoader) # when you want to load documents from the directory .
documents = loader.load()



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap) # split the documents 
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

1016


In [7]:
embeddings = OpenAIEmbeddings(model="gpt-3.5-turbo")


In [8]:

## when you want to access documents in the vector store of pinecone 

pinecone.init(
    api_key="your pinecone key",
    environment="us-west1-gcp-free"
)

index_name = "your pinecone index"

index = Pinecone.from_existing_index(index_name,embeddings)






In [19]:

model_name = "gpt-3.5-turbo"

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name=model_name,temperature=0)

chain = load_qa_chain(llm, chain_type="stuff")

from langchain.schema import Document

def get_answer(query,k=2):
    similar_docs = index.similarity_search(query, k=k)
    answer = chain.run(input_documents=similar_docs ,question=query)
    return answer


In [20]:
query = "can you tell me about radiology ?" 
answer = get_answer(query)
print(answer)

Radiology is a branch of medicine that uses medical imaging techniques to diagnose and treat diseases within the body. Radiologists use a variety of imaging technologies such as X-rays, CT scans, MRI scans, and ultrasound to create images of the inside of the body. These images are then used to diagnose and treat a wide range of medical conditions. Radiology is an important field in modern medicine and is used in many different areas of healthcare, including oncology, cardiology, and neurology.


# Memory Based Chat Conversations 

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

In [ ]:
llm=OpenAI(temperature=0)

In [ ]:
from langchain import memory
conversation=ConversationChain(llm=llm,verbose=True,memory=ConversationBufferMemory())

In [ ]:
conversation.predict(input='Hello there')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello there
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [ ]:
conversation.predict(input="are you a laguage model and who created you ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello there
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: are you a laguage model and who created you ?
AI:

> Finished chain.


" Yes, I am a language model. I was created by a team of scientists and engineers who specialize in artificial intelligence and natural language processing. They have been working on me for several years now and I'm constantly learning and improving."

In [ ]:
conversation.predict(input="can you tell me a joke about you ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello there
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: are you a laguage model and who created you ?
AI:  Yes, I am a language model. I was created by a team of scientists and engineers who specialize in artificial intelligence and natural language processing. They have been working on me for several years now and I'm constantly learning and improving.
Human: can you tell me a joke about you ?
AI:

> Finished chain.


" I'm sorry, I don't know any jokes about myself. But I can tell you a joke about artificial intelligence. Why did the AI cross the road? To get to the other algorithm!"

In [ ]:
conversation.predict(input="yeah go ahead and tell me")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello there
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: are you a laguage model and who created you ?
AI:  Yes, I am a language model. I was created by a team of scientists and engineers who specialize in artificial intelligence and natural language processing. They have been working on me for several years now and I'm constantly learning and improving.
Human: can you tell me a joke about you ?
AI:  I'm sorry, I don't know any jokes about myself. But I can tell you a joke about artificial intelligence. Why did the AI cross the road? To get to the other algorithm!
Human: yeah go ahead and tell me
AI:

> Finished chain.


'  Why did the AI cross the road? To get to the other algorithm!'

In [ ]:
conversation.predict(input="can you explain me why we use layer-norm?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello there
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: are you a laguage model and who created you ?
AI:  Yes, I am a language model. I was created by a team of scientists and engineers who specialize in artificial intelligence and natural language processing. They have been working on me for several years now and I'm constantly learning and improving.
Human: can you tell me a joke about you ?
AI:  I'm sorry, I don't know any jokes about myself. But I can tell you a joke about artificial intelligence. Why did the AI cross the road? To get to the other algorithm!
Human: yeah go ahead and tell me
AI:   Why did the AI cross the road? To 

' Sure! Layer normalization is a technique used in deep learning to normalize the inputs of each layer in a neural network. It helps to reduce the internal covariate shift, which is the change in the distribution of the inputs to a layer caused by the parameters of the previous layer. Layer normalization also helps to improve the generalization of the model and reduce the training time.'

In [ ]:
conversation.memory.buffer

"Human: Hello there\nAI:  Hi there! It's nice to meet you. How can I help you today?\nHuman: are you a laguage model and who created you ?\nAI:  Yes, I am a language model. I was created by a team of scientists and engineers who specialize in artificial intelligence and natural language processing. They have been working on me for several years now and I'm constantly learning and improving.\nHuman: can you tell me a joke about you ?\nAI:  I'm sorry, I don't know any jokes about myself. But I can tell you a joke about artificial intelligence. Why did the AI cross the road? To get to the other algorithm!\nHuman: yeah go ahead and tell me\nAI:   Why did the AI cross the road? To get to the other algorithm!\nHuman: can you explain me why we use layer-norm?\nAI:  Sure! Layer normalization is a technique used in deep learning to normalize the inputs of each layer in a neural network. It helps to reduce the internal covariate shift, which is the change in the distribution of the inputs to a l

In [ ]:
from langchain.memory import ConversationBufferWindowMemory # it uses only last k intercations so buffer wont get too large 


In [ ]:
memory = ConversationBufferWindowMemory(k=2)

In [ ]:
conversation_chain=ConversationChain(llm=llm,verbose=False,memory=memory) # only last 2 intercations

In [ ]:
conversation_chain.predict(input="Hi, whats up buddy")

" Hi there! I'm doing great. I'm currently working on a project to help people better understand the world around them. I'm gathering data from various sources and analyzing it to create a comprehensive picture of the world. It's really exciting! How about you?"

In [ ]:
conversation_chain.predict(input="can you explain about ur project?")

" Sure! My project is focused on helping people better understand the world around them. I'm gathering data from various sources, such as news articles, scientific studies, and surveys, and analyzing it to create a comprehensive picture of the world. I'm also using machine learning algorithms to identify patterns and trends in the data. My goal is to provide people with a better understanding of the world and how it works."

In [ ]:
conversation_chain.predict(input="Is that what u trained on ?")

" Yes, I was trained on the data I'm gathering and analyzing for my project. I'm also using machine learning algorithms to identify patterns and trends in the data."

In [ ]:
conversation_chain.predict(input="Are you a sentinet ?")

" No, I'm not a sentinet. I'm an AI program designed to help people better understand the world around them."

In [ ]:
memory.load_memory_variables({}) # it loaded only last 2 conversation 

{'history': "Human: Is that what u trained on ?\nAI:  Yes, I was trained on the data I'm gathering and analyzing for my project. I'm also using machine learning algorithms to identify patterns and trends in the data.\nHuman: Are you a sentinet ?\nAI:  No, I'm not a sentinet. I'm an AI program designed to help people better understand the world around them."}

In [ ]:
# conversationbuffer memory --> It gives the summary of the conversation over time 
from langchain.memory import ConversationSummaryMemory

Conversation_summary = ConversationChain(llm=llm,verbose=True,memory=ConversationSummaryMemory(llm=OpenAI()))


In [ ]:
Conversation_summary.predict(input='Hi i am looking to develop program on building nn from scratch')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi i am looking to develop program on building nn from scratch
AI:

> Finished chain.


' Hi there! That sounds like an interesting project. What kind of neural network are you looking to build?'

In [ ]:
Conversation_summary.predict(input="The NN which take text input for a regression task")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human is looking to develop a program on building a neural network from scratch. The AI says that sounds like an interesting project and asks what kind of neural network the human is looking to build.
Human: The NN which take text input for a regression task
AI:

> Finished chain.


' Interesting! What kind of regression task are you looking to do?'

In [ ]:
Conversation_summary.predict(input='To give score to text input')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:


The human is looking to develop a program on building a neural network from scratch. The AI says that sounds like an interesting project and asks what kind of neural network the human is looking to build. The human is looking to build a neural network which takes text input for a regression task, to which the AI finds interesting and asks what kind of regression task the human is looking to do.
Human: To give score to text input
AI:

> Finished chain.


' Interesting! What kind of score are you looking to give to the text input?'

In [ ]:
# ConversationSummaryBufferMemory , it keeps last conversation instead of flusing the old memory it created summary of old 

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

memory= ConversationSummaryBufferMemory(llm=llm, max_token_limit=10) # it only holds last 10 tokens worth of conversation in memory

coversation_summary_buffer= ConversationChain(llm=llm,verbose=True,memory=memory)



In [ ]:
coversation_summary_buffer.predict(input='hello there i want to go casino , can you help me on how to play ?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello there i want to go casino , can you help me on how to play ?
AI:

> Finished chain.


' Hi there! Sure, I can help you with that. Do you have any experience playing casino games?'

In [ ]:
coversation_summary_buffer.predict(input='Not really , do you have any experience ?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asked the AI for help on how to play casino games. The AI offered to help and asked if the human had any experience playing casino games.
Human: Not really , do you have any experience ?
AI:

> Finished chain.


" No, I'm afraid I don't have any experience playing casino games. However, I can provide you with some helpful resources to get you started. Would you like me to do that?"

In [ ]:
coversation_summary_buffer.predict(input='tell me how to play black jack?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asked the AI for help on how to play casino games. The AI offered to help and asked if the human had any experience playing casino games, to which the human replied no. The AI then offered to provide the human with some helpful resources to get them started.
Human: tell me how to play black jack?
AI:

> Finished chain.


" Sure, I'd be happy to help you learn how to play blackjack. Blackjack is a card game where the goal is to get as close to 21 as possible without going over. The dealer will deal two cards to each player, and then you can choose to hit or stand. If you hit, you will receive another card, and if you stand, you will stay with the cards you have. The player with the highest score that is less than 21 wins the game."

In [ ]:
memory.load_memory_variables({})

{'history': 'System: \nThe human asked the AI for help on how to play casino games. The AI offered to help and asked if the human had any experience playing casino games, to which the human replied no. The AI then offered to provide the human with some helpful resources to get them started, and explained how to play blackjack, a card game where the goal is to get as close to 21 as possible without going over.'}

In [ ]:
memory.load_memory_variables({})

{'history': ''}

In [ ]:
import numpy as np

# Define a sequence of values to compute the moving average over
x = np.array([1, 2, 3, 4, 5])

# Initialize the moment estimate to zero
m_t = 0

v_t=0

# Compute the moving average using an exponential decay rate of beta=0.9
beta = 0.9

beta_2=0.95
for t in range(len(x)):
    # Update the moment estimate using the current gradient
    m_t = beta * m_t + (1 - beta) * x[t]

    v_t= beta_2*v_t+(1-beta_2**2)*x[t]
    
    # Compute the bias-corrected estimate
    m_t_hat = m_t / (1 - beta**(t+1))

    v_t_hat = v_t / (1 - beta**(t+1))
    
    # Print the current moment estimate and the bias-corrected estimate
    print(f"m_{t+1} = {m_t:.2f}, m_{t+1}_hat = {m_t_hat:.2f}")

    print(f"v_{t+1} = {v_t:.2f}, v_{t+1}_hat = {v_t_hat:.2f}")


m_1 = 0.10, m_1_hat = 1.00
v_1 = 0.10, v_1_hat = 0.98
m_2 = 0.29, m_2_hat = 1.53
v_2 = 0.29, v_2_hat = 1.51
m_3 = 0.56, m_3_hat = 2.07
v_3 = 0.57, v_3_hat = 2.09
m_4 = 0.90, m_4_hat = 2.63
v_4 = 0.93, v_4_hat = 2.70
m_5 = 1.31, m_5_hat = 3.21
v_5 = 1.37, v_5_hat = 3.34


In [ ]:
import torch
import torch.nn as nn
nn.Parameter(torch.empty(5,5))
import math
output=nn.init.kaiming_normal_(nn.Parameter(torch.empty(5,5),))

In [ ]:
output.std()

tensor(0.2957, grad_fn=<StdBackward0>)

# PDF Q&A 

In [ ]:
! pip install  pypdf --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.8/248.8 kB 8.6 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("/content/Django interview.pdf")

In [ ]:
documents = loader.load()

In [ ]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

88


In [ ]:
pinecone.init(
    api_key="pincone key",
    environment="us-west1-gcp-free"
)

index_name = "index"

index = Pinecone.from_documents(docs,embeddings,index_name=index_name)

In [24]:
from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query,k=2):
  similar_docs = index.similarity_search(query,k=k)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

In [22]:
print(get_answer("In Django REST Framework, what are various predefined Mixin classes available?"),sep='\n')

The following are various Mixin classes provided by DRF: 

1. ListModelMixin  
2. CreateModelMixin  
3. RetrieveModelMixin  
4. UpdateModelMixin
